# DC Link Voltage Calibration Routine #  
- SJ 7/17/22

Monitor DC Link voltage externally and compare against ADC counts to create scalar or curve fit

## Setup System & Connect via Ethernet ##

In [ ]:
import os
import pathlib as pl
import sys
import time
import pandas as pd
import matplotlib.pyplot as plt

repo_dir = pl.Path(os.path.dirname(os.getcwd()))
sys.path.append(str(repo_dir / 'AMDC-Firmware' / 'scripts'))

from AMDC import AMDC
from AMDC_Logger import AMDC_Logger, find_mapfile
from AMDC_LivePlot import AMDC_LivePlot

amdc = AMDC()

# After creating the amdc object...

# Set comm defaults for Ethernet
amdc.setup_comm_defaults('eth')

# Init ethernet 
amdc.eth_init()

# Set up the default ASCII command socket
s0, s0_id = amdc.eth_new_socket('ascii_cmd')
amdc.eth_set_default_ascii_cmd_socket(s0)

# Cretae AMDC Logger Object
mapfile_path = find_mapfile(repo_dir)
logger = AMDC_Logger(AMDC = amdc, mapfile = mapfile_path)

### Procedure ###

- ADC measurement already has 0.00125 V/count scalar applied in code
- measure Vdc at various voltages
    - 0, 50, 100, 150, 200, 250 - Ran out of PSU

LV ADC board: 0.00125 V/count


## Vdc Link Measure ##

- Measured Voltage : Counts
    -   0.0 : -0.263
    -  50.0 : 120.00
        - 0.41666666
    - 100.0 : 240.07
        - 0.41654517
    - 150.0 : 360.55
        - 0.41603106
    - 199.9 : 480.81
        - 0.41575674
    - 249.9 : 600.92
        - 0.41586234

In [ ]:
# Setup Logger
logger.sync()

# Register Vars
logger.register('Vdc', samples_per_sec = 2000, var_type = 'double')

# Clear log
logger.empty_all()

# Define Command
# chn 0 phase U active high
# chn 1 phase V active high
# chn 2 phase W active high
# chn 3 All half rail
# chn 4 All ground
cmd_str = 'gen_ctrl chn 4'
amdc.cmd(cmd_str)

# Define Command
cmd_str = 'gen_ctrl init'
amdc.cmd(cmd_str)

# Record zero curr data for 5 seconds
logger.log(duration = 20)

# Define Command
cmd_str = 'gen_ctrl deinit'
amdc.cmd(cmd_str)

# Retrieve Data - return as Panda DataFrame obj
data = logger.dump(
    log_vars = 'Vdc',
    file = 'calData/voltageCalibration_250v.csv',
    comment = ' ')

# Unregister Vars
logger.unregister_all()
data.describe().to_csv('calData/voltageCal_250v_describe.csv')

In [ ]:
data.Vdc.describe()

In [ ]:
data.Vdc.plot()

## Shutdown and Clear Logs

In [ ]:
# Define Command
cmd_str = 'gen_ctrl deinit'
# Send command
amdc.cmd(cmd_str)

In [ ]:
# Unregister Vars
logger.unregister_all()